In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from natsort import natsorted

from mtcnn import MTCNN

2023-11-17 02:46:37.954285: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 02:46:38.788554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
vid_path = os.listdir('./input/')[0]

In [4]:
def FrameCap(path):
    if not os.path.exists('./frames'):
        os.makedirs('frames')
    vid = cv2.VideoCapture(path)
    cnt = 0
    flag = 1
    while flag:
        flag, image = vid.read()
        if image is not None:
            cv2.imwrite(f'./frames/frame{cnt}.png', image)
            cnt += 1
        else:
            flag = 0

In [5]:
FrameCap('./input' + os.sep + vid_path)

In [6]:
frame_list = natsorted(os.listdir("./frames/"))

In [7]:
detector = MTCNN()

2023-11-17 02:48:39.683183: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-17 02:48:39.730282: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-17 02:48:39.730410: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
def fivePoints(filename):
    img = cv2.imread(filename)
    detection = detector.detect_faces(img)[0]
    landmarks = detection["keypoints"]
    eye_l = list(landmarks["left_eye"])
    eye_r = list(landmarks["right_eye"])
    mouth_l = list(landmarks["mouth_left"])
    mouth_r = list(landmarks["mouth_right"])
    nose = list(landmarks["nose"])
    return eye_l, eye_r, mouth_l, mouth_r, nose, img

In [9]:
base = fivePoints('./frames' + os.sep + frame_list[0])

2023-11-17 02:48:42.385704: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


1/1 [==============================] - 3s 3s/step


2023-11-17 02:48:43.557680: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 0s 209ms/step


In [10]:
base = np.matrix([np.float32(base[0]), np.float32(base[1]), np.float32(base[2])])

In [11]:
if not os.path.exists('./aligned/'):
    os.makedirs('aligned')
for item in frame_list:
    try:
        landmarks = fivePoints('./frames/' + os.sep + item)
        img = landmarks[5]
        landmarks = np.matrix([np.float32(landmarks[0]), np.float32(landmarks[1]), np.float32(landmarks[2])])
        M = cv2.getAffineTransform(landmarks, base)
        rows, cols, ch = img.shape
        new_img = cv2.warpAffine(img, M, (cols, rows))
        cv2.imwrite('./aligned/' + os.sep + item, new_img)
    except:
        print(item)

4/4 [==============================] - 0s 2ms/step
frame48.png
1/1 [==============================] - 0s 15ms/step


In [13]:
aligned = []
for filename in natsorted(os.listdir('./aligned/')):
    img = cv2.imread('./aligned' + os.sep + filename)
    aligned.append(img)

out = cv2.VideoWriter('aligned.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, (cols, rows))
for i in range(len(aligned)):
    out.write(aligned[i])
    
out.release()